# Application: Longitudinal Vehicle Model

#### References

---

**Remark**

This notebook has been inspired from Coursera's course Introduction to Self-Driving Cars 

---

In this notebook, we will implement the forward longitudinal vehicle model. The model accepts throttle inputs and steps through the longitudinal dynamic equations. Once implemented, you will be given a set of inputs that drives over a small road slope to test your model.

The input to the model is a throttle percentage $x_\theta \in [0,1]$ which provides torque to the engine and subsequently accelerates the vehicle for forward motion. 

The dynamic equations consist of many stages to convert throttle inputs to wheel speed (engine -> torque converter -> transmission -> wheel). These stages are bundled together in a single inertia term $J_e$ which is used in the following combined engine dynamic equations.


\begin{align}
    J_e \dot{\omega}_e &= T_e - (GR)(r_{eff} F_{load}) \\ m\ddot{x} &= F_x - F_{load}
\end{align}

Where $T_e$ is the engine torque, $GR$ is the gear ratio, $r_{eff}$ is the effective radius, $m$ is the vehicle mass, $x$ is the vehicle position, $F_x$ is the tire force, and $F_{load}$ is the total load force. 

The engine torque is computed from the throttle input and the engine angular velocity $\omega_e$ using a simplified quadratic model. 

\begin{align}
    T_e = x_{\theta}(a_0 + a_1 \omega_e + a_2 \omega_e^2)
\end{align}

The load forces consist of aerodynamic drag $F_{aero}$, rolling friction $R_x$, and gravitational force $F_g$ from an incline at angle $\alpha$. The aerodynamic drag is a quadratic model and the friction is a linear model.


\begin{align}
    F_{load} &= F_{aero} + R_x + F_g \\
    F_{aero} &= \frac{1}{2} C_a \rho A \dot{x}^2 = c_a \dot{x}^2\\
    R_x &= N(\hat{c}_{r,0} + \hat{c}_{r,1}|\dot{x}| + \hat{c}_{r,2}\dot{x}^2) \approx c_{r,1} \dot{x}\\
    F_g &= mg\sin{\alpha}
\end{align}


Note that the absolute value is ignored for friction since the model is used for only forward motion ($\dot{x} \ge 0$).  The tire force is computed using the engine speed and wheel slip equations.

\begin{align}
    \omega_w &= (GR)\omega_e \\
    s &= \frac{\omega_w r_e - \dot{x}}{\dot{x}}\\
    F_x &= \left\{\begin{array}{lr}
        cs, &  |s| < 1\\
        F_{max}, & \text{otherwise}
        \end{array}\right\} 
\end{align}

Where $\omega_w$ is the wheel angular velocity and $s$ is the slip ratio. 

We setup the longitudinal model inside a Python class below. The vehicle begins with an initial velocity of 5 m/s and engine speed of 100 rad/s. All the relevant parameters are defined and like the bicycle model, a sampling time of 10ms is used for numerical integration.

In [2]:
# Need these so that Jupyter can find the simulator types
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from vehicle.vehicle_base import VehicleBase
from dynamics.dynamics_base import DynamicsBase
from systems.system_state import SysState
from ode_integrators.forward_euler import ODEScalarFWDEuler

In [ ]:
class EngineDynamics():
    
    def __init__(self, sample_time):
        self.__integrator=ODEScalarFWDEuler(step_size=sample_time)
        
    def execute(self, **kwargs):
        
        c_a  = kwargs['c_a']
        
        pass
        

In [4]:
class VehicleDynamics(DynamicsBase):
    
    def __init__(self, sample_time):
        DynamicsBase.__init__(self)
        self.__state = SysState(n_entries=5)
        self.__state.add_state("x", 0)
        self.__state.add_state("v", 1) #= 5
        self.__state.add_state("a", 2) #= 0
        self.__state.add_state("w_e",3) #= 100
        self.__state.add_state("w_e_dot",4)  #= 0)
        
        # integrator for the velocity
        self.set_integrator(self, "velocity", integrator=ODEScalarFWDEuler(step_size=sample_time))
    
    @property
    def state(self):
        return self.__state

    @state.setter
    def state(self, value):
        self.__state = value
        
    def execute(self, **kwargs):
        
        c_a  = kwargs['c_a']
        
        pass

In [10]:
class Vehicle(VehicleBase):
    
    def __init__(self, properties, sample_time):
        VehicleBase.__init__(self, properties)
        self.__dynamics = VehicleDynamics(sample_time=sample_time)
        
    @property
    def state(self):
        return self.__dynamics.state

    @state.setter
    def state(self, value):
        self.__dynamics.state = value

    def get_old_state(self, name, idx):
        return self.__dynamics.state.get_old_state(name=name, idx=idx)

    def set_old_state(self, name, idx, value):
        self.__dynamics.state.set_old_state(name=name, idx=idx, value=value)
        
    def execute(self, **kwargs):
        
        self.get_property("propulsion").execute(**kwargs)
        
        self.__dynamics.execute(**kwargs)

In [11]:
### SIMULATION DRIVER

sample_time = 0.01
time_end = 100

t_data = np.arange(0,time_end,sample_time)
v_data = np.zeros_like(t_data)


# Gear ratio, effective radius, mass + inertia
vehicle_properties = {"a_0":400, "a_1":0.1, "a_2":-0.0002,
                      "GR":0.35, "r_e":0.3, "J_e":10, "m": 2000, "c_a":1.36, 
                      "propulsion": EngineDynamics(sample_time=sample_time) }


model = Vehicle(properties=vehicle_properties, sample_time=sample_time)


# throttle percentage between 0 and 1
throttle = 0.2

# incline angle (in radians)
alpha = 0

kwargs = {"throttle":0.2, "alpha": 0.0, "c_a":model.get_property("c_a")}


# the simulation time
time = 0.0
for i in range(t_data.shape[0]):
    
    print("At time: %f"%time)
    
    #v_data[i] = model.v
    model.execute(**kwargs)
    
    time += sample_time
    
plt.plot(t_data, v_data)
plt.show()

At time: 0.000000
At time: 0.010000
At time: 0.020000
At time: 0.030000
At time: 0.040000
At time: 0.050000
At time: 0.060000
At time: 0.070000
At time: 0.080000
At time: 0.090000
At time: 0.100000
At time: 0.110000
At time: 0.120000
At time: 0.130000
At time: 0.140000
At time: 0.150000
At time: 0.160000
At time: 0.170000
At time: 0.180000
At time: 0.190000
At time: 0.200000
At time: 0.210000
At time: 0.220000
At time: 0.230000
At time: 0.240000
At time: 0.250000
At time: 0.260000
At time: 0.270000
At time: 0.280000
At time: 0.290000
At time: 0.300000
At time: 0.310000
At time: 0.320000
At time: 0.330000
At time: 0.340000
At time: 0.350000
At time: 0.360000
At time: 0.370000
At time: 0.380000
At time: 0.390000
At time: 0.400000
At time: 0.410000
At time: 0.420000
At time: 0.430000
At time: 0.440000
At time: 0.450000
At time: 0.460000
At time: 0.470000
At time: 0.480000
At time: 0.490000
At time: 0.500000
At time: 0.510000
At time: 0.520000
At time: 0.530000
At time: 0.540000
At time: 0

At time: 14.120000
At time: 14.130000
At time: 14.140000
At time: 14.150000
At time: 14.160000
At time: 14.170000
At time: 14.180000
At time: 14.190000
At time: 14.200000
At time: 14.210000
At time: 14.220000
At time: 14.230000
At time: 14.240000
At time: 14.250000
At time: 14.260000
At time: 14.270000
At time: 14.280000
At time: 14.290000
At time: 14.300000
At time: 14.310000
At time: 14.320000
At time: 14.330000
At time: 14.340000
At time: 14.350000
At time: 14.360000
At time: 14.370000
At time: 14.380000
At time: 14.390000
At time: 14.400000
At time: 14.410000
At time: 14.420000
At time: 14.430000
At time: 14.440000
At time: 14.450000
At time: 14.460000
At time: 14.470000
At time: 14.480000
At time: 14.490000
At time: 14.500000
At time: 14.510000
At time: 14.520000
At time: 14.530000
At time: 14.540000
At time: 14.550000
At time: 14.560000
At time: 14.570000
At time: 14.580000
At time: 14.590000
At time: 14.600000
At time: 14.610000
At time: 14.620000
At time: 14.630000
At time: 14.

At time: 29.110000
At time: 29.120000
At time: 29.130000
At time: 29.140000
At time: 29.150000
At time: 29.160000
At time: 29.170000
At time: 29.180000
At time: 29.190000
At time: 29.200000
At time: 29.210000
At time: 29.220000
At time: 29.230000
At time: 29.240000
At time: 29.250000
At time: 29.260000
At time: 29.270000
At time: 29.280000
At time: 29.290000
At time: 29.300000
At time: 29.310000
At time: 29.320000
At time: 29.330000
At time: 29.340000
At time: 29.350000
At time: 29.360000
At time: 29.370000
At time: 29.380000
At time: 29.390000
At time: 29.400000
At time: 29.410000
At time: 29.420000
At time: 29.430000
At time: 29.440000
At time: 29.450000
At time: 29.460000
At time: 29.470000
At time: 29.480000
At time: 29.490000
At time: 29.500000
At time: 29.510000
At time: 29.520000
At time: 29.530000
At time: 29.540000
At time: 29.550000
At time: 29.560000
At time: 29.570000
At time: 29.580000
At time: 29.590000
At time: 29.600000
At time: 29.610000
At time: 29.620000
At time: 29.

At time: 44.110000
At time: 44.120000
At time: 44.130000
At time: 44.140000
At time: 44.150000
At time: 44.160000
At time: 44.170000
At time: 44.180000
At time: 44.190000
At time: 44.200000
At time: 44.210000
At time: 44.220000
At time: 44.230000
At time: 44.240000
At time: 44.250000
At time: 44.260000
At time: 44.270000
At time: 44.280000
At time: 44.290000
At time: 44.300000
At time: 44.310000
At time: 44.320000
At time: 44.330000
At time: 44.340000
At time: 44.350000
At time: 44.360000
At time: 44.370000
At time: 44.380000
At time: 44.390000
At time: 44.400000
At time: 44.410000
At time: 44.420000
At time: 44.430000
At time: 44.440000
At time: 44.450000
At time: 44.460000
At time: 44.470000
At time: 44.480000
At time: 44.490000
At time: 44.500000
At time: 44.510000
At time: 44.520000
At time: 44.530000
At time: 44.540000
At time: 44.550000
At time: 44.560000
At time: 44.570000
At time: 44.580000
At time: 44.590000
At time: 44.600000
At time: 44.610000
At time: 44.620000
At time: 44.

At time: 59.100000
At time: 59.110000
At time: 59.120000
At time: 59.130000
At time: 59.140000
At time: 59.150000
At time: 59.160000
At time: 59.170000
At time: 59.180000
At time: 59.190000
At time: 59.200000
At time: 59.210000
At time: 59.220000
At time: 59.230000
At time: 59.240000
At time: 59.250000
At time: 59.260000
At time: 59.270000
At time: 59.280000
At time: 59.290000
At time: 59.300000
At time: 59.310000
At time: 59.320000
At time: 59.330000
At time: 59.340000
At time: 59.350000
At time: 59.360000
At time: 59.370000
At time: 59.380000
At time: 59.390000
At time: 59.400000
At time: 59.410000
At time: 59.420000
At time: 59.430000
At time: 59.440000
At time: 59.450000
At time: 59.460000
At time: 59.470000
At time: 59.480000
At time: 59.490000
At time: 59.500000
At time: 59.510000
At time: 59.520000
At time: 59.530000
At time: 59.540000
At time: 59.550000
At time: 59.560000
At time: 59.570000
At time: 59.580000
At time: 59.590000
At time: 59.600000
At time: 59.610000
At time: 59.

At time: 79.100000
At time: 79.110000
At time: 79.120000
At time: 79.130000
At time: 79.140000
At time: 79.150000
At time: 79.160000
At time: 79.170000
At time: 79.180000
At time: 79.190000
At time: 79.200000
At time: 79.210000
At time: 79.220000
At time: 79.230000
At time: 79.240000
At time: 79.250000
At time: 79.260000
At time: 79.270000
At time: 79.280000
At time: 79.290000
At time: 79.300000
At time: 79.310000
At time: 79.320000
At time: 79.330000
At time: 79.340000
At time: 79.350000
At time: 79.360000
At time: 79.370000
At time: 79.380000
At time: 79.390000
At time: 79.400000
At time: 79.410000
At time: 79.420000
At time: 79.430000
At time: 79.440000
At time: 79.450000
At time: 79.460000
At time: 79.470000
At time: 79.480000
At time: 79.490000
At time: 79.500000
At time: 79.510000
At time: 79.520000
At time: 79.530000
At time: 79.540000
At time: 79.550000
At time: 79.560000
At time: 79.570000
At time: 79.580000
At time: 79.590000
At time: 79.600000
At time: 79.610000
At time: 79.

At time: 94.090000
At time: 94.100000
At time: 94.110000
At time: 94.120000
At time: 94.130000
At time: 94.140000
At time: 94.150000
At time: 94.160000
At time: 94.170000
At time: 94.180000
At time: 94.190000
At time: 94.200000
At time: 94.210000
At time: 94.220000
At time: 94.230000
At time: 94.240000
At time: 94.250000
At time: 94.260000
At time: 94.270000
At time: 94.280000
At time: 94.290000
At time: 94.300000
At time: 94.310000
At time: 94.320000
At time: 94.330000
At time: 94.340000
At time: 94.350000
At time: 94.360000
At time: 94.370000
At time: 94.380000
At time: 94.390000
At time: 94.400000
At time: 94.410000
At time: 94.420000
At time: 94.430000
At time: 94.440000
At time: 94.450000
At time: 94.460000
At time: 94.470000
At time: 94.480000
At time: 94.490000
At time: 94.500000
At time: 94.510000
At time: 94.520000
At time: 94.530000
At time: 94.540000
At time: 94.550000
At time: 94.560000
At time: 94.570000
At time: 94.580000
At time: 94.590000
At time: 94.600000
At time: 94.

NameError: name 'plt' is not defined